In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")

In [3]:
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [4]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

C:\Users\sinha\AppData\Local\Temp\ipykernel_40712\564156087.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the `langchain-neo4j package and should be used instead. To use it run `pip install -U `langchain-neo4j` and import as `from `langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)


In [5]:
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""
graph.query(moview_query)

[]

In [6]:
groq_api_key=os.getenv("GROQ_API_KEY")

In [7]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")
llm

C:\Users\sinha\PycharmProjects\complete-genai\15-GraphDB\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001AAF92C4050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AAF92C4D70>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_neo4j import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    llm=llm, graph=graph, verbose=True,
    allow_dangerous_requests=True
)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000001AAC9BF8980>, cypher_generation_chain=PromptTemplate(input_variables=['examples', 'question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nExamples (optional):\n{examples}\n\nThe question is:\n{question}')
| RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': Fa

In [9]:
chain.graph_schema

'Node properties:\nMovie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}\nPerson {name: STRING}\nGenre {name: STRING}\nRelationship properties:\n\nThe relationships:\n(:Movie)-[:IN_GENRE]->(:Genre)\n(:Person)-[:DIRECTED]->(:Movie)\n(:Person)-[:ACTED_IN]->(:Movie)'

In [10]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [28]:
examples_fixed = [
    { "question": ex["question"], "query": ex["query"] }
    for ex in examples[:5]
]
examples_fixed

[{'question': 'How many artists are there?',
  'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'},
 {'question': 'Which actors played in the movie Casino?',
  'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"},
 {'question': 'How many movies has Tom Hanks acted in?',
  'query': "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)"},
 {'question': "List all the genres of the movie Schindler's List",
  'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"},
 {'question': 'Which actors have worked in movies from both the comedy and action genres?',
  'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}]

In [33]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["question", "query"],
    template="Question: {question}\nCypher: {query}"
)

prompt = FewShotPromptTemplate(
    examples=examples_fixed,  # <-- use cleaned examples
    example_prompt=example_prompt,
    prefix=(
        "You are a Neo4j expert. "
        "Given an input question, create a syntactically accurate Cypher query.\n"
        "Schema:\n{schema}\n"
    ),
    suffix="User input: {question}\nCypher query:",
    input_variables=["question", "schema"]
)


In [34]:
prompt

FewShotPromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}], example_prompt=PromptTemplate(inp

In [35]:
print(prompt.input_variables)

['question', 'schema']


In [39]:
chain.invoke("Which actors played in the movie Casino?")



> Entering new GraphCypherQAChain chain...


[#EB36]  _: <CONNECTION> error: Failed to write data to connection ResolvedIPv4Address(('34.126.114.186', 7687)) (ResolvedIPv4Address(('34.126.114.186', 7687))): ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
Unable to retrieve routing information
Transaction failed and will be retried in 1.0427297232215498s (Unable to retrieve routing information)


Generated Cypher:
MATCH (m:Movie {title: "Casino"})-[:ACTED_IN]->(p:Person) RETURN p.name


[#EB37]  _: <CONNECTION> error: Failed to write data to connection IPv4Address(('si-041b3fbb-c4f4.production-orch-0066.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.114.186', 7687))): ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
Transaction failed and will be retried in 2.1964788883124355s (Failed to write data to connection IPv4Address(('si-041b3fbb-c4f4.production-orch-0066.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.114.186', 7687))))


Full Context:
[]

> Finished chain.


{'query': 'Which actors played in the movie Casino?',
 'result': "I don't know the answer."}

In [40]:
chain.invoke("actors who acted in multiple movies")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(b:Person) WHERE a<>b RETURN a.name AS actor, COUNT(DISTINCT m) AS count
Full Context:
[{'actor': 'Tim Allen', 'count': 2}, {'actor': 'Tom Hanks', 'count': 2}, {'actor': 'Don Rickles', 'count': 1}, {'actor': 'Jim Varney', 'count': 1}, {'actor': 'Bradley Pierce', 'count': 1}, {'actor': 'Kirsten Dunst', 'count': 1}, {'actor': 'Jonathan Hyde', 'count': 1}, {'actor': 'Robin Williams', 'count': 2}, {'actor': 'Ann-Margret', 'count': 1}, {'actor': 'Jack Lemmon', 'count': 1}]

> Finished chain.


{'query': 'actors who acted in multiple movies',
 'result': 'Tim Allen, Tom Hanks, Robin Williams acted in multiple movies.'}